In [1]:
import dask, concurrent.futures, time, warnings, os, re, pickle
from osgeo import gdal
import os
import requests as r
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio as rio
import rioxarray as riox
import time
import xarray as xr
from urllib.request import urlopen
from xml.etree.ElementTree import parse,fromstring
from pandas import to_datetime
from rasterio.crs import CRS
from datetime import datetime, timedelta
from netrc import netrc
from pyproj import Proj
from src.hls_funcs import fetch
from src.hls_funcs.masks import mask_hls, shp2mask, bolton_mask
from src.hls_funcs.indices import ndvi_func
from src.hls_funcs.smooth import smooth_xr, despike_ts_xr
import cartopy.crs as ccrs
from rasterio.plot import show
from src.hls_funcs.predict import pred_bm, pred_bm_se, pred_cov
import dask.diagnostics
import matplotlib.pyplot as plt

In [2]:
#wkDIR = '/project/cper_neon_aop/hls_nrt/'
#outDIR = '/90daydata/cper_neon_aop/hls_nrt/' + prefix

wkDIR = os.getcwd()
outDIR_base = 'data/hls_nrt/'

cluster_loc = 'local'

In [3]:
yr = 2022

#prefix = 'redtop' 
prefix = 'cper'

if prefix == 'cper': 
    aoi_f = os.path.join('data/ground/cper_pastures_2017_clip.shp')
    df_aoi = gpd.read_file(aoi_f)
    subunit_name_old = 'Past_Name_'
    subunit_name = 'Pasture'
elif prefix == 'redtop':
    from src.utils.convert import kmz_to_shp
    df_aoi = kmz_to_shp('data/ground/RedTop_Boundary.kmz', 'data/ground/')
    df_aoi = df_aoi.to_crs(epsg=32613)
    subunit_name_old = None
    subunit_name = None

outDIR = os.path.join(outDIR_base, prefix)
    
mod_bm = pickle.load(open('src/models/CPER_HLS_to_VOR_biomass_model_lr_simp.pk', 'rb'))
mod_cov = pickle.load(open('src/models/CPER_HLS_to_LPI_cover_pls_binned_model.pk', 'rb'))

/home/spkearney/miniconda3/envs/hls_nrt_env/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator PLSRegression from version 0.24.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
os.chdir(wkDIR)
print(yr)
t00 = time.time()
t0 = time.time()
t1 = time.time()
if cluster_loc == 'local':
    print('   setting up Local cluster...')
    from dask.distributed import LocalCluster, Client
    import dask
    aws=False
    fetch.setup_env(aws=aws)
    cluster = LocalCluster(n_workers=8, threads_per_worker=2)
    client = Client(cluster)
    display(client)
elif cluster_loc == 'coiled':
    import coiled
    aws=True
    fetch.setup_env(aws=aws)
    s3_cred = fetch.setup_netrc(creds=['spkearney', '1mrChamu'], aws=aws)
    coiled.create_software_environment(
    name="hls_cog_coiled",
    conda="hls_cog_coiled_env.yaml")
    cluster = coiled.Cluster(
        name="hls_cog_coiled",
        software="kearney-sp/hls_cog_coiled",
        n_workers=5,
        worker_cpu=2,
        scheduler_cpu=2,
        backend_options={"region": "us-west-2"},
        environ=dict(GDAL_DISABLE_READDIR_ON_OPEN='FALSE', 
                   #AWS_NO_SIGN_REQUEST='YES',
                   GDAL_MAX_RAW_BLOCK_CACHE_SIZE='200000000',
                   GDAL_SWATH_SIZE='200000000',
                   VSI_CURL_CACHE_SIZE='200000000',
                   CPL_VSIL_CURL_ALLOWED_EXTENSIONS='TIF',
                   GDAL_HTTP_UNSAFESSL='YES',
                   GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/cookies.txt'),
                   GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/cookies.txt'),
                   AWS_REGION='us-west-2',
                   AWS_SECRET_ACCESS_KEY=s3_cred['secretAccessKey'],
                   AWS_ACCESS_KEY_ID=s3_cred['accessKeyId'],
                   AWS_SESSION_TOKEN=s3_cred['sessionToken'])
    )
    client = Client(cluster)
    display(client)
elif cluster_loc == 'hpc':
    from dask.distributed import LocalCluster, Client
    import dask_jobqueue as jq
    import dask
    from jupyter_server import serverapp
    
    # get the server address for porting
    try:
        jupServer = [x for x in serverapp.list_running_servers()][0]
    except IndexError:
        # manually copy/paste the server address
        jupServer = {'base_url': '/node/ceres19-compute-98-eth.scinet.local/17710/'}
    print('   setting up cluster on HPC...')
    aws=False
    fetch.setup_env(aws=aws)
    dask.config.set({'distributed.dashboard.link': jupServer['base_url'] + 'proxy/{port}/status'})
    partition='short',#'short','debug', 'mem', 'mem-low',
    num_processes = 4
    num_threads_per_processes = 2
    mem = 2.5*num_processes*num_threads_per_processes
    n_cores_per_job = num_processes*num_threads_per_processes
    clust = jq.SLURMCluster(queue=partition,
                            processes=num_processes,
                            cores=n_cores_per_job,
                            memory=str(mem)+'GB',
                            interface='ib0',
                            #interface='enp24s0f0',
                            local_directory='$TMPDIR',
                            death_timeout=30,
                            walltime='02:00:00',
                            job_extra=["--output=/dev/null","--error=/dev/null"])
    client=Client(clust)
    #Scale Cluster 
    num_jobs=16
    clust.scale(jobs=num_jobs)
    try:
        client.wait_for_workers(n_workers=num_jobs*num_processes, timeout=60)
    except dask.distributed.TimeoutError as e:
        print(str(num_jobs*num_processes) + ' workers not available. Continuing with available workers.')
        #print(e)
        pass
    display(client)
print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

if not os.path.exists(outDIR):
    os.mkdir(outDIR)

if subunit_name_old is not None:
    df_aoi = df_aoi.rename(columns={subunit_name_old: subunit_name})
if subunit_name is not None:
    df_aoi = df_aoi.dissolve(by=subunit_name).reset_index()

start_date = str(yr - 1) + "-11-01"
end_date = str(yr + 1) + "-03-01"

# set the date range for analysis
date_rng = pd.date_range(start=start_date, end=end_date)
date_rng = date_rng[date_rng <= datetime.today()]

2022
   setting up Local cluster...


2022-06-15 18:34:19,088 - distributed.diskutils - INFO - Found stale lock file and directory '/mnt/c/Users/TBGPEA-Sean/git_repos/hls_nrt/dask-worker-space/worker-1ldzd_n2', purging
2022-06-15 18:34:19,096 - distributed.diskutils - INFO - Found stale lock file and directory '/mnt/c/Users/TBGPEA-Sean/git_repos/hls_nrt/dask-worker-space/worker-6_q95oi1', purging
2022-06-15 18:34:19,103 - distributed.diskutils - INFO - Found stale lock file and directory '/mnt/c/Users/TBGPEA-Sean/git_repos/hls_nrt/dask-worker-space/worker-7c1d_nkg', purging
2022-06-15 18:34:19,112 - distributed.diskutils - INFO - Found stale lock file and directory '/mnt/c/Users/TBGPEA-Sean/git_repos/hls_nrt/dask-worker-space/worker-kylswk5b', purging
2022-06-15 18:34:19,123 - distributed.diskutils - INFO - Found stale lock file and directory '/mnt/c/Users/TBGPEA-Sean/git_repos/hls_nrt/dask-worker-space/worker-m2c49fr3', purging
2022-06-15 18:34:19,136 - distributed.diskutils - INFO - Found stale lock file and directory '/

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 16,Total memory: 15.50 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34773,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 15.50 GiB
Comm: tcp://127.0.0.1:34555,Total threads: 2
Dashboard: http://127.0.0.1:42765/status,Memory: 1.94 GiB
Nanny: tcp://127.0.0.1:46181,


...completed in 1.0 secs
total elasped time: 0.02 mins



In [6]:
print('   fetching data...')
t1 = time.time()
# fetch the COG data from the AWS bucket
data_dict = {'date_range': [str(date_rng.min().date()), str(date_rng.max().date())]}
hls_ds = fetch.get_hls(hls_data=data_dict,
                       bbox=df_aoi.total_bounds, 
                       proj_epsg=df_aoi.crs.to_epsg(),
                       lim=500,
                       aws=aws)
hls_ds = hls_ds.where(hls_ds['eo:cloud_cover'] != None, drop=True)
hls_ds = hls_ds.where(hls_ds['eo:cloud_cover'] < 90, drop=True)
#hls_ds = hls_ds.loc[dict(x=slice(517587.0, 527283.0), y=slice(4524402.0, 4514699.0))].chunk({'y': -1,
#                                                                                             'x': -1,
#                                                                                             'time':1})
#hls_mask = mask_hls(hls_ds['FMASK'])
idx_load_try = 0
max_load_try = 5
while idx_load_try < max_load_try:
    try:
        hls_ds = hls_ds.load()#.where(hls_mask == 0).load()
        idx_load_try = max_load_try
    except RuntimeError as e:            
        if e.args[0] == 'Set changed size during iteration':
            print('Warning: error with cluster set size. Restarting cluster and retrying ' + \
              str(idx_load_try+1) + ' of ' + str(max_load_try))
            clust.close()
            client.close()
            clust = jq.SLURMCluster(queue=partition,
                            processes=num_processes,
                            cores=n_cores_per_job,
                            memory=str(mem)+'GB',
                            interface='ib0',
                            #interface='enp24s0f0',
                            local_directory='$TMPDIR',
                            death_timeout=30,
                            walltime='02:00:00',
                            job_extra=["--output=/dev/null","--error=/dev/null"])
            client=Client(clust)
            #Scale Cluster 
            clust.scale(jobs=num_jobs)
            try:
                client.wait_for_workers(n_workers=num_jobs*num_processes, timeout=60)
            except dask.distributed.TimeoutError as e:
                print(str(num_jobs*num_processes) + ' workers not available. Continuing with available workers.')
                #print(e)
                pass
            display(client)
        else:
            print('Warning: error connecting to lpdaac. Retrying ' + str(idx_load_try+1) + ' of ' + str(max_load_try))
        idx_load_try += 1
    except rio.errors.RasterioIOError as e:
        print('Warning: error loading data. Retrying ' + str(idx_load_try+1) + ' of ' + str(max_load_try))
        idx_load_try += 1

#hls_ds['time'] = hls_ds['time'].dt.date
print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

   fetching data...
...completed in 189.0 secs
total elasped time: 4.09 mins



In [7]:
hls_ds = hls_ds.reset_coords(drop=True)

In [8]:
if len(np.unique(hls_ds.values)) < len(hls_ds.time.values):
    print('    reducing along id dimension to single observation for each date')
    #hls_ds = hls_ds.chunk({'time': 1, 'y': 50, 'x': 50})
    t1 = time.time()
    hls_ds = hls_ds.groupby('time').mean().compute()
    print(hls_ds)
    print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
    print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

    reducing along id dimension to single observation for each date
<xarray.Dataset>
Dimensions:  (time: 74, y: 323, x: 323)
Coordinates:
  * x        (x) float64 5.176e+05 5.176e+05 5.176e+05 ... 5.272e+05 5.272e+05
  * y        (y) float64 4.524e+06 4.524e+06 4.524e+06 ... 4.515e+06 4.515e+06
  * time     (time) object 2021-11-03 2021-11-04 ... 2022-06-12 2022-06-14
Data variables:
    BLUE     (time, y, x) float64 718.0 744.0 741.0 ... 1.079e+03 1.025e+03
    GREEN    (time, y, x) float64 1.053e+03 1.078e+03 ... 1.508e+03 1.453e+03
    RED      (time, y, x) float64 1.39e+03 1.432e+03 ... 1.822e+03 1.765e+03
    NIR1     (time, y, x) float64 2.023e+03 2.132e+03 ... 3e+03 2.937e+03
    SWIR1    (time, y, x) float64 2.468e+03 2.711e+03 ... 4.112e+03 4.018e+03
    SWIR2    (time, y, x) float64 1.721e+03 1.903e+03 ... 3.162e+03 3.063e+03
    FMASK    (time, y, x) float64 64.0 64.0 64.0 64.0 ... 128.0 128.0 128.0
    SZA      (time, y, x) float64 5.815e+03 5.815e+03 ... 2.37e+03 2.37e+03


In [9]:
hls_ds['time'] = pd.to_datetime(hls_ds.time)

In [10]:
if not os.path.exists(os.path.join(outDIR, prefix + '_hls_tmp/')):
        os.mkdir(os.path.join(outDIR, prefix + '_hls_tmp/'))
hls_ds.to_netcdf(os.path.join(outDIR,
                              prefix + 
                              '_hls_tmp/hls_ds_' + 
                              str(yr) +'_' + 
                              datetime.now().strftime('%Y%m%d') + '.nc'))

In [11]:
print(hls_ds)

<xarray.Dataset>
Dimensions:  (time: 74, y: 323, x: 323)
Coordinates:
  * x        (x) float64 5.176e+05 5.176e+05 5.176e+05 ... 5.272e+05 5.272e+05
  * y        (y) float64 4.524e+06 4.524e+06 4.524e+06 ... 4.515e+06 4.515e+06
  * time     (time) datetime64[ns] 2021-11-03 2021-11-04 ... 2022-06-14
Data variables:
    BLUE     (time, y, x) float64 718.0 744.0 741.0 ... 1.079e+03 1.025e+03
    GREEN    (time, y, x) float64 1.053e+03 1.078e+03 ... 1.508e+03 1.453e+03
    RED      (time, y, x) float64 1.39e+03 1.432e+03 ... 1.822e+03 1.765e+03
    NIR1     (time, y, x) float64 2.023e+03 2.132e+03 ... 3e+03 2.937e+03
    SWIR1    (time, y, x) float64 2.468e+03 2.711e+03 ... 4.112e+03 4.018e+03
    SWIR2    (time, y, x) float64 1.721e+03 1.903e+03 ... 3.162e+03 3.063e+03
    FMASK    (time, y, x) float64 64.0 64.0 64.0 64.0 ... 128.0 128.0 128.0
    SZA      (time, y, x) float64 5.815e+03 5.815e+03 ... 2.37e+03 2.37e+03
    SAA      (time, y, x) float64 1.614e+04 1.614e+04 ... 1.318e+04 1.3

2022-06-15 18:42:42,081 - distributed.nanny - WARNING - Restarting worker
2022-06-15 18:42:42,113 - distributed.nanny - WARNING - Restarting worker
2022-06-15 18:42:42,144 - distributed.nanny - WARNING - Restarting worker
2022-06-15 18:42:42,151 - distributed.nanny - WARNING - Restarting worker
2022-06-15 18:42:42,160 - distributed.nanny - WARNING - Restarting worker
2022-06-15 18:42:42,166 - distributed.nanny - WARNING - Restarting worker
2022-06-15 18:42:42,172 - distributed.nanny - WARNING - Restarting worker
2022-06-15 18:42:42,182 - distributed.nanny - WARNING - Restarting worker
2022-06-15 18:42:42,262 - distributed.nanny - WARNING - Restarting worker
2022-06-15 18:42:42,263 - distributed.nanny - WARNING - Restarting worker
2022-06-15 18:42:42,266 - distributed.nanny - WARNING - Restarting worker
2022-06-15 18:42:42,269 - distributed.nanny - WARNING - Restarting worker
2022-06-15 18:42:42,273 - distributed.nanny - WARNING - Restarting worker
2022-06-15 18:42:42,277 - distributed.